# Named Entities Recognition

In [1]:
import os
import nltk
import json
import re
import codecs
import string
import subprocess
import unicodedata
import collections
import pandas as pd


from nltk import ne_chunk
from polyglot.text import Text

from nltk import word_tokenize
from collections import defaultdict
from polyglot.detect import Detector
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger
from nltk.internals import find_jars_within_path

from polyglot.downloader import downloader
#downloader.download("embeddings2.en","ner2.en")



# Important paths to access data

In [2]:
directory_name = os.path.dirname('/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt')
directory_name

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt'

In [3]:
sample_file = os.path.join('/home/alice','/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt')
sample_file

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt'

In [4]:
path = ("/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/")
path

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/'

In [5]:
# We will work with files in this directory
os.path.basename('/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/')

''

In [6]:
# this can differenciate a directory and a file

os.path.split('/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt')

('/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt',
 'Afghanistan 0 18Mar2015.txt')

##  Data storage customization

In [7]:
## Set important paths

BASE = os.path.join(os.path.dirname('/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt'))
BASE

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt'

In [8]:
DOCS = os.path.join("scraped_data", "files_in_txt")
DOCS

'scraped_data/files_in_txt'

In [9]:
path = BASE
path

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt'

In [24]:
# list of files we have.
#os.listdir(path)

In [12]:
address = os.listdir(path)

In [13]:
len(address)

1260

In [14]:
path

'/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt'

In [15]:
DOCS

'scraped_data/files_in_txt'

# Data description:
### Our target files are in  txt  format .


In [16]:
def get_documents(path=BASE):
    
    '''Total number of txt file we have in corpus folder'''
    
    for name in os.listdir(path):
        if name.startswith('') and name.endswith('.txt'):
            yield os.path.join(path, name)
print(len(list(get_documents())))

1260


# NLTK corpus 

In [17]:
CORPUS = os.path.join(BASE, "", "corpus")

In [18]:
path = "/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/"

In [19]:
## Create a path to extract the corpus.

CORPUS = os.path.join("/home/alice/Documents/Project/info_extraction/scraped_data", "corpus")

In [20]:
#fname = os.path.splitext(os.path.basename("/home/alice/Documents/Project/info_extraction/scraped_data/Malawi MDRMW012 06Dec2016"))[0] + ".txt"
fname =' /home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt'

In [22]:
# sample file 

example_file = '/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt'
print(example_file)

/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt


In [23]:
#example_file = '/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Tanzania MDRTZ016 10Mar2015.txt'

In [24]:
'''get the number of strings compose sample file before filtering'''
document = open(example_file, 'r').read()
print(len(document))

9014


In [25]:
'''Remaining string after filtering'''
document = filter(
            lambda char: char in string.printable,
            unicodedata.normalize('NFKD', document)
        )
len(list(document)) #   number of remaining strings after filt

8990

In [26]:
# The filter removed 24 = 9014-8990 special characters 

In [27]:
corpus = '/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/corpus/'

In [28]:
#outpath = os.path.join(corpus, fname)
outpath = '/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/corpus/'


### How to read a single report , remove nonprintable characters by  NRKF unicode and write the fitered report on outpth

In [29]:
## For a sample file

In [30]:
def sample_corpus(BASE,CORPUS):
    #for path in get_documents(BASE):
        example_file = '/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/Afghanistan 0 18Mar2015.txt'
        document = open(example_file, 'r').read()
        print(len(document))
        document = filter(
            lambda char: char in string.printable,
            unicodedata.normalize('NFKD', document)
        )
        document = ''.join(list(document))
        print(document[:100])
        fname = os.path.basename(example_file)
        print(fname)
        outpath = os.path.join(CORPUS, fname)
        print(outpath)

        with codecs.open(outpath, 'w') as f:
            f.write(document)
    

In [31]:
sample_corpus(BASE, CORPUS)

9014
Information Bulletin n 1
Glide n AV-2015-000016-AFG
18 March 2015
This bulletin is being issued
for 
Afghanistan 0 18Mar2015.txt
/home/alice/Documents/Project/info_extraction/scraped_data/corpus/Afghanistan 0 18Mar2015.txt


#  TOP 25 LINES Filtered corpus

WHY DID WE CONSIDERED FIRST 25 FIVE LINES ONLY?

In [32]:
## WHY DID WE CONSIDERED FIRST 25 FIVE LINES ONLY?
#=================================================
#1.##  -For most of the pdf reports from IFRC, there is a "small table" containing important data,  
#2.##  - the txt files gotten    from conversion  of PDF made  that "small table"  into almost  25 first lines of each txt document.
#3.##  -I SPLIT THOSE TOP 25 LINES FOR EACH REPORT I COLLECTED THEM AS NEW CORCUP FOLDER CALLED "initial_lines_txt"

In [33]:

def all_corpus(BASE,CORPUS):
    '''Returns filtered corpus for the whole folder '''

    for path in get_documents(BASE):
        #print(path)
        document = open(path, 'r').read()
        #print(len(document))
        document = filter(
            lambda char: char in string.printable,
            unicodedata.normalize('NFKD', document.encode('utf-8').decode('utf-8'))
        )
        document = ''.join(document)
        document = "\n".join(document.split('\n')[:25])
        fname = os.path.basename(path)
        outpath = os.path.join(
            "/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt", 
            fname)

        with codecs.open(outpath, 'w') as f:
            f.write(document)
    

In [34]:
all_corpus(path,CORPUS)

### Customizable NLTK corpus to access txt data on the disk an load the data

In [35]:
kddcorpus = nltk.corpus.PlaintextCorpusReader(
    "/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt",  
    '.*\.txt')  # For a full text :"/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt",
kddcorpus

<PlaintextCorpusReader in '/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt'>

In [36]:
# Just a look on top 15 documents

initial_ids = kddcorpus.fileids()[:15]; initial_ids

['- Global - MAA00001 22Jul2015.txt',
 '- Global - MAA00006 24Apr2015.txt',
 '- Global - MAA00010 10Nov2015.txt',
 '- Global - MAA00021 02Jun2015.txt',
 '- Global - MAA00028 01May2015.txt',
 '- Global - MAA00029 21Jun2016.txt',
 '- Global - MAA00040 02Jun2015.txt',
 '- Global - MAA00040 10Nov2015.txt',
 '- Global - MAA00040 12May2015.txt',
 '- Global - MDR00001 01Jun2016.txt',
 '- Global - MDR00001 21Apr2015.txt',
 '- Global - MDR42003 01Jun2016.txt',
 '- Global - MDR42003 02Feb2016.txt',
 '- Global - MDR42003 04Mar2016.txt',
 '- Global - MDR42003 22Jun2016.txt']

### FreqDist to calculate sum of words we have for corpus and stopwords

In [37]:
'''How many words and stopwords in our corpus ? '''

words = nltk.FreqDist(kddcorpus.words())
count = sum(words.values())
vocab = len(words)

print("Corpus contains a vocabulary of {} and a word count of {}.".format(
    count, vocab
))

Corpus contains a vocabulary of 343611 and a word count of 11843.


In [38]:
#sample file on position 29 called 'Africa regional office MDR60002 03Nov2015.txt'

fids = kddcorpus.fileids()[28];fids

'Africa regional office MDR60002 03Nov2015.txt'

# NERC Tools  to the corpus 

#### For this Research we are using starnford model  version 2014  but later we will use 2016 version for comparaison.


In [39]:
'''    
   Download jar and model of stanford of version 2014 from the NLP website
   - Return a disctionary of location, persons, organization and others 
    '''
# root of model, jar and stanford packages.
root  = os.path.expanduser('~/models/stanford-ner-2014-01-04/')
# later We will uroot = os.path.expanduser('~/models/stanford-corenlp-full-2016-10-31/') 


model = os.path.join(root, 'classifiers/english.muc.7class.distsim.crf.ser.gz')


jar  = os.path.join(root, 'stanford-ner-2014-01-04.jar')

# STANFORD NER FOR TOP 25 LINES

In [40]:
def stanford_entities(fileids=None, corpus=kddcorpus):
    
    jar = "/home/alice/models/stanford-ner-2014-01-04/stanford-ner-2014-01-04.jar"
    model = "/home/alice/models/stanford-ner-2014-01-04/classifiers/english.muc.7class.distsim.crf.ser.gz"
    
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()
    tagger  = StanfordNERTagger(model, jar)

    for fileid in fileids:
        text  = corpus.words(fileid)

        chunk = []

        for token, tag in tagger.tag(text):
            if tag == 'O':
                if chunk:
                    # Flush the current chunk
                    etext =  " ".join([c[0] for c in chunk])
                    etag  = chunk[0][1]
                    chunk = []

                    if etag == 'PERSON':
                        key = 'persons'
                    elif etag == 'ORGANIZATION':
                        key = 'organizations'
                    elif etag == 'LOCATION':
                        key = 'locations'
                    else:
                        key = 'other'

                    results[fileid][key].append(etext)

            else:
                chunk.append((token, tag)) # Build chunk from tags

    return results

In [25]:
#stanford_entities(fileids=None, corpus=kddcorpus)

In [42]:
stanalgo = stanford_entities(fileids=None, corpus=kddcorpus)
pd.DataFrame(stanalgo)

,- Global - MAA00001 22Jul2015.txt,- Global - MAA00006 24Apr2015.txt,- Global - MAA00010 10Nov2015.txt,- Global - MAA00021 02Jun2015.txt,- Global - MAA00028 01May2015.txt,- Global - MAA00029 21Jun2016.txt,- Global - MAA00040 02Jun2015.txt,- Global - MAA00040 10Nov2015.txt,- Global - MAA00040 12May2015.txt,- Global - MDR00001 01Jun2016.txt,...,Zimbabwe MDRZW010 26Aug2015.txt,Zimbabwe MDRZW011 04Oct2016.txt,Zimbabwe MDRZW011 09May2016.txt,Zimbabwe MDRZW011 15Oct2015.txt,Zimbabwe MDRZW011 16Sep2015.txt,Zimbabwe MDRZW011 18Sep2015.txt,Zimbabwe MDRZW011 24Mar2016.txt,Zimbabwe MDRZW011 27Apr2016.txt,Zimbabwe MDRZW011 29Sep2015.txt,table_report.txt
locations,[Neonatal],"[Geneva, Geneva]",[Bolivia],"[Sendai, Japan, Geneva, Cali, Colombia]","[Geneva, Panama, Kuala Lumpur, Nairobi, Dubai,...","[Syria, Iraq, Afghanistan, Libya, Ukraine, Yem...",NaN,NaN,NaN,NaN,...,"[Zimbabwe, Zimbabwe, Manicaland, Mashonaland C...","[Zimbabwe, Mashonaland East, Mashonaland West]","[Mashonaland East, Mashonaland Central, Zimbab...","[Mashonaland East, Mashonaland Central, Zimbab...",NaN,NaN,"[Southern Africa, Mashonaland East, Mashonalan...",[South of Zimbabwe],"[Southern Africa, Mashonaland East, Mashonalan...","[Philippines, Myanmar, Myanmar, Myanmar, Iraq,..."
organizations,[Global Health Report Health Department 2014 T...,"[National Societies, NSKD, International Feder...","[DREF 2013 Number Amount, Red Cross Red Cresce...","[Preparatory Committee of WCDRR, DRR, Fourth G...","[IFRC Global Logistics Service, IFRC, National...","[Red Cross, Red Crescent, IFRC, IFRC, Middle E...","[Federation of Red Cross, Rules for Disaster R...","[IFRC, Crisis Management Department ( DCM ) Gl...","[International Federation of Red Cross, Nation...","[DREF 2015 Number Amount, Red Cross Red Cresce...",...,"[Danish Red Cross, Norwegian Red Cross, Finnis...","[IFRC, Southern Africa NS, Host National Socie...","[Disaster Risk Management Coordinator, IFRC, S...","[Disaster Risk Management Coordinator, IFRC, S...","[NET Appeal, This Emergency Appeal, Zimbabwe R...","[FedNet, Inkind Goods & Transport Inkind Perso...","[EA Operations, Disaster Risk Management Coord...","[Emergency Appeal, Zimbabwe Red Cross Society,...","[Disaster Risk Management Coordinator, IFRC, Z...",[Iraq]
other,NaN,"[2014, 2014, 2014 The Difference Overview The ...","[31 %, 69 %, 2014, 2 per cent, 2013, April 201...","[2015, March 2015, July, November 2014, June, ...","[2015, Strategy 2020, 2015, 2014, 2015, 2014, ...","[2014, 2014, 1990, 7 %, 2014]",NaN,"[January June 2015, January 2015 12 months 72]",NaN,"[1241 19, 33 %, 67 %, 2015, 2014, 2014, 2015]",...,"[2015, August , 2015, December , 2014, January...","[September 2016, September 2015 16 September 2...","[April 2016, October 2015, 25 April 2016, Sept...","[October 2015, August 2015, September 2015, Se...","[October, December 2015, September 2015 83, Ma...",NaN,"[October 2015, 14 March 2016, September 2015 1...","[September 2015, April 2016, December 2016, 30...","[September 2015, September 2015, September , 2...",NaN
persons,NaN,NaN,NaN,NaN,[Sierra Leone],"[Jaime Sepulveda, Christopher Murray]",NaN,"[Pankaj Mishra, Hakan Karay]",NaN,NaN,...,NaN,"[Jamie LeSueur, Maxwell Phiri]",[Maxwell Phiri],[Maxwell Phiri],NaN,NaN,[Maxwell Phiri],NaN,[Maxwell Phiri],"[Typhoon Nock, Typhoon Nock, Typhoon Nock, Noc..."


# POLYGLOT ENTITIES

In [43]:
def polyglot_entities(fileids=None,  section = None, corpus=kddcorpus):
    """
     Another way of extracting entities from the corpus
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()


    for fileid in fileids:
        #print(fileid)
        
        if section is not None:
            raw_text = list(sectpull([fileid],section=section))[0][1]
        else:
            raw_text = corpus.raw(fileid)
        raw_text = ''.join(x for x in raw_text if x not in string.printable[-2:])
        text = Text(raw_text)

        for entity in text.entities:
            etext = " ".join(entity)

            if entity.tag == 'I-PER':
                key = 'persons'
                
            elif entity.tag == 'I-ORG':
                key = 'organizations'
            elif entity.tag == 'Location_Names':
                key = 'I-locations'
            else:
                key = 'other'

            results[fileid][key].append(etext)

    return results

In [26]:
#polyglot_entities(fileids=None, section = None, corpus=kddcorpus)

In [45]:
''''''#polyglot_entities(fileids=None, section = None, corpus=kddcorpus)

''

###  POLYGLOT TABULAR  REPRESENTATION

In [46]:
polyalgo = polyglot_entities(fileids=None, section = None, corpus=kddcorpus)
pd.DataFrame(polyalgo)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


,- Global - MAA00001 22Jul2015.txt,- Global - MAA00006 24Apr2015.txt,- Global - MAA00010 10Nov2015.txt,- Global - MAA00021 02Jun2015.txt,- Global - MAA00028 01May2015.txt,- Global - MAA00029 21Jun2016.txt,- Global - MAA00040 02Jun2015.txt,- Global - MAA00040 10Nov2015.txt,- Global - MAA00040 12May2015.txt,- Global - MDR00001 01Jun2016.txt,...,Zimbabwe MDRZW010 26Aug2015.txt,Zimbabwe MDRZW011 04Oct2016.txt,Zimbabwe MDRZW011 09May2016.txt,Zimbabwe MDRZW011 15Oct2015.txt,Zimbabwe MDRZW011 16Sep2015.txt,Zimbabwe MDRZW011 18Sep2015.txt,Zimbabwe MDRZW011 24Mar2016.txt,Zimbabwe MDRZW011 27Apr2016.txt,Zimbabwe MDRZW011 29Sep2015.txt,table_report.txt
organizations,"[Health, First, Adolescent, Sanitation, Cross,...","[Global, National Society and Knowledge Develo...","[Crescent, Cross, Red Crescent Societies, Disa...","[World Conference, UNISDR, WCDRR, Community Re...","[Logistics, Global Logistics Service, National...",[Global Health],"[nternational, of Red, Red Crescent, Managemen...","[Crisis Management Department, Crisis Management]","[Federation of Red, Red Crescent, Management, ...","[Crescent, Cross, Red Crescent Societies, Nati...",...,"[Society :, Society : Zimbabwe Red Cross, Nati...","[Society, Zimbabwe Red Cross Society, Provincial]","[National Society, Zimbabwe Red Cross Society ...","[National Society, Zimbabwe Red Cross Society ...","[Food, Zimbabwe Red Cross Society]","[Goods, Transport, Cross, British Red Cross, B...","[EA, Society, Zimbabwe Red Cross Society Headq...","[Appeal, Zimbabwe Red Cross Society]","[Society, Zimbabwe Red Cross Society, Mashonal...","[Population, Population, Costa Rica MDRCR014, ..."
other,NaN,"[Geneva, Geneva]","[Bolivia, Bolivia]","[Sendai, Japan, Geneva, Cali , Colombia, Cali]","[Geneva, Panama, Kuala Lumpur, Nairobi, Dubai,...","[Syria, Iraq, Afghanistan, Libya, Ukraine, Yem...",NaN,NaN,NaN,NaN,...,"[Society : Zimbabwe, Cross, Zimbabwe, Mashonal...","[Zimbabwe, IFRC, Southern Africa, Zimbabwe, Ma...","[Southern Africa, Zimbabwe, Mashonaland East, ...","[Southern Africa, Zimbabwe, Mashonaland East, ...",[Zimbabwe],"[Beit Trust, Netherlands, Netherlands Government]","[Southern Africa, Zimbabwe, Mashonaland East, ...","[South, Zimbabwe]","[Southern Africa, Zimbabwe Red Cross, Zimbabwe...","[Philippines, Philippines, Philippines, Myanma..."
persons,NaN,NaN,NaN,[DRR],"[GLS, GLS, GLS]","[Jaime Sepulveda, Christopher Murray]",NaN,"[Simon Eccleshall, Pankaj Mishra, Hakan Karay,...",NaN,NaN,...,[IFRC SARO],"[Jamie LeSueur, Maxwell Phiri, ZRCS]","[Naemi, Maxwell Phiri, IFRC, Federations DREF]","[Naemi, Maxwell Phiri, IFRC]",NaN,NaN,"[Naemi, Maxwell Phiri, IFRC]",NaN,"[Naemi, Maxwell Phiri, IFRC]","[Typhoon Nock, MDRDO009, Typhoon Nock, Haiyan,..."


In [47]:
#all_kddcorpus = polyglot_entities(fileids=None, section = None, corpus=kddcorpus)

# NLTK ENTITIES

In [48]:
def nltk_entities(fileids=None, section = None,corpus=kddcorpus):
    """
    Extract entities using the NLTK named entity chunker.
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()

    for fileid in fileids:
        if section is not None:
            text = nltk.pos_tag(nltk.word_tokenize(list(sectpull([fileid],section=section))[0][1]))
        else:
            text = nltk.pos_tag(corpus.words(fileid))



        for entity in nltk.ne_chunk(text):
            if isinstance(entity, nltk.tree.Tree):
                etext = " ".join([word for word, tag in entity.leaves()])
                label = entity.label()
            else:
                continue

            if label == 'PERSON':
                key = 'persons'
            elif label == 'ORGANIZATION':
                key = 'organizations'
            elif label == 'LOCATION':
                key = 'locations'
            elif label == 'GPE':
                key = 'other'
            else:
                key = None

            if key:
                results[fileid][key].append(etext)

    return results

In [27]:
#nltk_entities(fileids=None, section = None,corpus=kddcorpus)

## NLTK ENTITIES TABULAR  REPRESENTATION

In [50]:
nltkalgo = nltk_entities(fileids=None, section = None,corpus=kddcorpus)
pd.DataFrame(nltkalgo)

,- Global - MAA00001 22Jul2015.txt,- Global - MAA00006 24Apr2015.txt,- Global - MAA00010 10Nov2015.txt,- Global - MAA00021 02Jun2015.txt,- Global - MAA00028 01May2015.txt,- Global - MAA00029 21Jun2016.txt,- Global - MAA00040 02Jun2015.txt,- Global - MAA00040 10Nov2015.txt,- Global - MAA00040 12May2015.txt,- Global - MDR00001 01Jun2016.txt,...,Zimbabwe MDRZW010 26Aug2015.txt,Zimbabwe MDRZW011 04Oct2016.txt,Zimbabwe MDRZW011 09May2016.txt,Zimbabwe MDRZW011 15Oct2015.txt,Zimbabwe MDRZW011 16Sep2015.txt,Zimbabwe MDRZW011 18Sep2015.txt,Zimbabwe MDRZW011 24Mar2016.txt,Zimbabwe MDRZW011 27Apr2016.txt,Zimbabwe MDRZW011 29Sep2015.txt,table_report.txt
locations,NaN,NaN,NaN,NaN,NaN,"[West Africa, West Africa, Caribbean]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[South],NaN,NaN
organizations,"[Global Health Report Health, Contents, CBHFA,...","[Global, Difference, National Society, NSKD, N...","[Overview Statistics, DREF, CHF Total, DREF, D...","[DRR, HFA2, DRR, WCDRR, WCDRR, HFA2, UNISDR, W...","[IFRC Global Logistics Service, GLS, IFRC, Nat...","[oPt, Ebola Virus Disease, EVD, Sahel, Horn, R...","[Red Cross, Red Crescent, MAA00040, DCMs, DCM,...","[DEVELOPMENT, UPDATE, INTERVENTION, DCM, CHF, ...","[International Federation, Red Cross, Red Cres...","[Overview Statistics, DREF, CHF Total, DREF, D...",...,"[MDRZW010, GLIDE, ZWE Date, Host National Soci...","[MDRZW011, Date, EPoA, IFRC, ZRCS, Host Nation...","[EPoA, IFRC, Host National Society, Branches, ...","[MDRZW011, EPoA, IFRC, Host National Society, ...","[FEWS, NET Appeal, MDRZW011, DREF, Zimbabwe Re...","[Public Folders, FedNet, Transport Inkind, Ame...","[EPoA, IFRC, Host National Society, Branches, ...","[MDRZW011, DREF, Zimbabwe Red Cross Society, Z...","[MDRZW011 Operations, EPoA, IFRC, Host Nationa...","[MDRPH023, MDRMM006, FileId, Population Moveme..."
other,"[Disease, Maternal, Neonatal, Child, Sanitation]","[Geneva, Long, Geneva]","[Bolivia, Bolivian]","[Sendai, Japan, Geneva, Cali, Colombia]","[Geneva, Panama, Dubai, Las Palmas, Ebola, Ira...","[Iraq, Afghanistan, Libya, Palestinian, Yemen,...",NaN,NaN,NaN,NaN,...,"[DREF, Issue, Norwegian, Mashonaland Central, ...","[Emergency, Zimbabwe, Mashonaland East, Mashon...","[Mashonaland East, Mashonaland Central, Britis...","[Mashonaland East, Mashonaland Central, Britis...","[Swiss, Swiss, Appeal, Swiss, Zimbabwes]","[British, British, Beit, Norwegian, Canadian, ...","[Mashonaland East, Mashonaland Central, Britis...","[Photo, Swiss, Swiss, Appeal, Swiss, Swiss, Sw...","[Mashonaland East, Mashonaland Central, Britis...","[Floods, Iraq, Iraq, Drought, Floods, Earthqua..."
persons,"[Annual, Annexes Annex, Health, First Aid]","[Knowledge Development Division, Term Planning...","[Start, Red Cross Red Crescent, Emergency Fund...","[Billion Coalition, Climate, Climate]","[Overview, Kuala Lumpur, Nairobi, Guinea, Arab]","[Latin America, Global Health, Jaime Sepulveda...","[Crisis Management, Rules, Disaster Relief, Gl...","[Disaster, Crisis Management Department, Globa...","[Crisis Management, Crisis Management, Disaste...","[Start, Red Cross Red Crescent, Disaster Relie...",...,"[Zimbabwe Red Cross Society, Danish Red Cross,...","[Jamie LeSueur, Operations Manager, Southern A...","[Emergency, Appeal, Disaster Risk, Southern Af...","[Emergency, Appeal Operation, Operations Updat...","[Food Security October, Appeal, Appeal, Zimbab...","[Emergency Appeal, Inkind Goods, Income Total,...","[Emergency, Appeal MDRZW011 EA Operations, Dis...","[Appeal, Appeal, Appeal, Appeal, Emergency App...","[Emergency, Appeal Operation, Disaster Risk, S...","[Philippines, Typhoon Nock, Active, Philippine..."


# Sample test for precision,  accuracy  and recall


In [58]:
#this testing file has been choosen randomly,and We retrieved personal names from it.
kddcorpus.fileids()[28]

'Africa regional office MDR60002 03Nov2015.txt'

In [59]:
# corpus sample path
fids =['/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt/Africa regional office MDR60002 03Nov2015.txt']

## We extracted  manually Personal names from 'Africa regional office MDR60002 03Nov2015.txt'   then  We  want to compare them to Personal names from NLTK, STANFORD and POLYGLOT 

In [60]:
'''stanford entities for person'''


'stanford entities for person'

In [61]:
def stanford_entities_test(fileids=None, corpus=kddcorpus):
    
    jar = "/home/alice/models/stanford-ner-2014-01-04/stanford-ner-2014-01-04.jar"
    model = "/home/alice/models/stanford-ner-2014-01-04/classifiers/english.muc.7class.distsim.crf.ser.gz"
    
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()
    tagger  = StanfordNERTagger(model, jar)

    for fileid in fileids:
        text  = corpus.words(fileid)

        chunk = []

        for token, tag in tagger.tag(text):
            if tag == 'O':
                if chunk:
                    # Flush the current chunk
                    etext =  " ".join([c[0] for c in chunk])
                    etag  = chunk[0][1]
                    chunk = []

                    if etag == 'PERSON':
                        key = 'persons'
                    
                    results[fileid][key].append(etext)

            else:
                chunk.append((token, tag)) # Build chunk from tags

    return results

In [62]:
#print(nltk_entities_test(fileids=fids, section = None,corpus=kddcorpus))

In [63]:
fids = ['/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt/Africa regional office MDR60002 03Nov2015.txt']

In [65]:
fids= ['/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt/Africa regional office MDR60002 03Nov2015.txt']
stanents = stanford_entities_test(fids)
stan_persons = {file: D['persons'] for file, D in stanents.items()}
print(stan_persons[fids[0]])
#len(stan_persons[fids[0]])

['Mamadou Basilah', 'Forecariah', 'Guinea', 'Ebola', 'Tommy Trenchard  IFRC Emergency', '2014', 'NGA', 'November 2015', 'Norbert Allale', 'Norbert Allale', 'Head of Ebola Regional Operations', 'Months Update Timeframe', 'August 2014 20 August 2015', 'August 2014', 'December 2015', '85 %', 'August 2014', 'January 2015', 'December 2015', 'Million Red Cross Red Crescent Movement', 'Liberian National Red Cross Society', 'Sierra Leone Red Cross Society', 'Danish Red Cross', 'French Red Cross', 'German Red Cross', 'Netherlands Red Cross', 'Spanish Red Cross', 'Swiss Red Cross', 'ICRC', 'UNICEF', 'CDC', 'Action Contre la Faim', 'Institut Pasteur Dakar', 'Catholic Relief Services', 'CAFOD', 'Caritas', 'Samaritans Purse']


###  Polyglot testing

In [66]:
def polyglot_entities_test(fileids=None,  section = None, corpus=kddcorpus):
    """
     Another way of extracting entities from the corpus
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()


    for fileid in fileids:
        #print(fileid)
        
        if section is not None:
            raw_text = list(sectpull([fileid],section=section))[0][1]
        else:
            raw_text = corpus.raw(fileid)
        raw_text = ''.join(x for x in raw_text if x not in string.printable[-2:])
        text = Text(raw_text)

        for entity in text.entities:
            etext = " ".join(entity)

            if entity.tag == 'I-PER':
                key = 'persons'

            results[fileid][key].append(etext)

    return results

In [67]:
polyents = polyglot_entities_test(fileids=fids, section = None, corpus=kddcorpus)

In [68]:
print(polyents[fids[0]])

defaultdict(<class 'list'>, {'persons': ['Mamadou Basilah', 'Klalantou', 'Forecariah', 'Guinea', 'Tommy Trenchard', 'SEN', 'Norbert Allale', 'Norbert Allale', 'Ebola', 'Operations', 'Crescent Movement', 'Guinea', 'Cross', 'Liberian', 'Red Cross Society', 'Sierra', 'Sierra Leone', 'Sierra Leone Red Cross Society', 'Cross', 'Cross', 'Cross', 'Netherlands', 'Cross', 'Cross', 'Cross', 'ICRC', 'of Health', 'WHO', 'UNICEF', 'CDC', 'Institut Pasteur Dakar', 'Catholic Relief Services', 'Concern', 'CARE', 'CAFOD', 'Caritas']})


### NLTK names of persons extraction

In [69]:
def nltk_entities_test(fileids=None, section = None,corpus=kddcorpus):
    """
    Extract entities using the NLTK named entity chunker.
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()

    for fileid in fileids:
        if section is not None:
            text = nltk.pos_tag(nltk.word_tokenize(list(sectpull([fileid],section=section))[0][1]))
        else:
            text = nltk.pos_tag(corpus.words(fileid))



        for entity in nltk.ne_chunk(text):
            if isinstance(entity, nltk.tree.Tree):
                etext = " ".join([word for word, tag in entity.leaves()])
                label = entity.label()
            else:
                continue

            if label == 'PERSON':
                key = 'persons'
            else:
                key = None

            if key:
                results[fileid][key].append(etext)

    return results

In [28]:
#nltkents = nltk_entities_test(fileids=fids, section = None,corpus=kddcorpus)
#nltkents

In [71]:
fids = ['/home/alice/Documents/Project/info_extraction/scraped_data/initial_lines_txt/Africa regional office MDR60002 03Nov2015.txt']
stanents = stanford_entities_test(fids)
print(stanents[fids[0]])

defaultdict(<class 'list'>, {'persons': ['Mamadou Basilah', 'Forecariah', 'Guinea', 'Ebola', 'Tommy Trenchard  IFRC Emergency', '2014', 'NGA', 'November 2015', 'Norbert Allale', 'Norbert Allale', 'Head of Ebola Regional Operations', 'Months Update Timeframe', 'August 2014 20 August 2015', 'August 2014', 'December 2015', '85 %', 'August 2014', 'January 2015', 'December 2015', 'Million Red Cross Red Crescent Movement', 'Liberian National Red Cross Society', 'Sierra Leone Red Cross Society', 'Danish Red Cross', 'French Red Cross', 'German Red Cross', 'Netherlands Red Cross', 'Spanish Red Cross', 'Swiss Red Cross', 'ICRC', 'UNICEF', 'CDC', 'Action Contre la Faim', 'Institut Pasteur Dakar', 'Catholic Relief Services', 'CAFOD', 'Caritas', 'Samaritans Purse']})


## Json file : I created manually a Json file contains person names from forst 25 lines of  a sample report" " in order to check accuracy,precision and recall.

In [72]:
manual_file = open('manual_file.json', 'r')
conn_string = json.load(manual_file)
conn_string

{'BothNames': {'1': 'Mamadou Basilah',
  '2': 'Tommy Trenchard',
  '3': 'Norbert Allale',
  '4': 'Caritas',
  '5': 'Samaritans'},
 'SingleName': {'1': 'Mamadou',
  '2': 'Basilah',
  '3': 'Tommy',
  '4': 'Trenchard',
  '5': 'Norbert',
  '6': 'Allale'}}

In [75]:
# ordered
manual = json.load(open('manual_file.json'))

In [76]:
'''
Creation of table for names of persons found manually polyentsand names of persons
extracted by NLTK,POLYGLOT AND STANFORD

'''



df1 = pd.Series(polyents[fids[0]]['persons'], index=None,
dtype=None, name='Polyglot NERC Authors', copy=False, fastpath=False)

df2=pd.Series([re.sub('\*',"",l) for l in stanents[fids[0]]['persons']],
index=None, dtype=None, name='Stanford NERC Authors', copy=False, fastpath=False)

df3=pd.Series([re.sub('\*',"",l) for l in nltkents[fids[0]]['persons']],
index=None, dtype=None, name='NLTKStandard NERC Authors', copy=False, fastpath=False)

df4 = pd.Series(manual['BothNames'], index=None,
dtype=None, name='Hand-labeled True BothNames', copy=False, fastpath=False)

met = pd.concat([df4,df3,df2,df1], axis=1).fillna('')
met

/home/alice/.local/lib/python3.4/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)
/home/alice/.local/lib/python3.4/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  result = result.union(other)


,Hand-labeled True BothNames,NLTKStandard NERC Authors,Stanford NERC Authors,Polyglot NERC Authors
1,Mamadou Basilah,,,
2,Tommy Trenchard,,,
3,Norbert Allale,,,
4,Caritas,,,
5,Samaritans,,,
0,,Mamadou Basilah,Mamadou Basilah,Mamadou Basilah
1,,Tommy Trenchard,Forecariah,Klalantou
2,,Norbert Allale Point,Guinea,Forecariah
3,,Norbert Allale,Ebola,Guinea
4,,Ebola Regional Operations,Tommy Trenchard IFRC Emergency,Tommy Trenchard


In [77]:
# Calculations and logic from http://www.kdnuggets.com/faq/precision-recall.html

def metrics(truth,run):
    truth = truth
    run = run
    TP = float(len(set(run) & set(truth)))

    if float(len(run)) >= float(TP):
        FP = len(run) - TP
    else:
        FP = TP - len(run)
    TN = 0
    if len(truth) >= len(run):
        FN = len(truth) - len(run)
    else:
        FN = 0

    accuracy = (float(TP)+float(TN))/float(len(truth))
    recall = (float(TP))/float(len(truth))
    precision = float(TP)/(float(FP)+float(TP))
    print ("The accuracy is %r" % accuracy)
    print ("The recall is %r" % recall)
    print ("The precision is %r" % precision)

    d = {'Predicted Negative': [TN,FN], 'Predicted Positive': [FP,TP]}
    metricsdf = pd.DataFrame(d, index=['Negative Cases','Positive Cases'])

    return metricsdf


In [78]:
metrics(polyents,nltkents) 

The accuracy is 1.0
The recall is 1.0
The precision is 1.0


,Predicted Negative,Predicted Positive
Negative Cases,0,0.0
Positive Cases,0,1.0


### Based on the table above,  some names of organization, abbreviations are classified as name of Persons, which cause the precision, recall and accuracy to be so small.

## Next step : 

### I will test for other categories of entities (location, organization and others )   then I will compare 3 ways of extracting  entities (nltk,polyglot and stanford) to entities found manually.




# 2nd PART

### CSV FILE FOR CTP REPORTS

In [80]:
data = pd.read_csv("/home/alice/Documents/Project/info_extraction/doc information all_s BIS_r2.csv", sep=None)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [82]:
data.head()

,Unnamed: 0,id.of.documents,country,appeal.code,date.of.document,CTP,Number.of.beneficiaries,CTP.budget,Number.of.CTP.beneficiaries,CTP.budget.source,...,Appeal,DREF,name,document.name,file.name,url,url.status,glide.number,type.of.event,date.of.event
0,1,- Global - 010198 01Jan1998.txt,- Global -,010198,01Jan1998,no,NaN,NaN,0.0,NaN,...,True,False,Disaster Relief Emergency Fund - appeal 1998 (...,Annual appeal,- Global - 010198 01Jan1998.pdf,http://www.ifrc.org/docs/appeals/annual98/0101...,OK,none,none,none
1,2,- Global - 010199 01Jan1999.txt,- Global -,010199,01Jan1999,no,NaN,NaN,0.0,NaN,...,True,False,Disaster Relief Emergency Fund - appeal 1999 (...,Annual appeal,- Global - 010199 01Jan1999.pdf,http://www.ifrc.org/docs/appeals/annual99/0101...,OK,none,none,none
2,3,- Global - 010298 01Jan1998.txt,- Global -,010298,01Jan1998,no,NaN,NaN,0.0,NaN,...,True,False,Global Telecommunications Network -appeal 1998...,Annual appeal,- Global - 010298 01Jan1998.pdf,http://www.ifrc.org/docs/appeals/annual98/0102...,OK,none,none,none
3,4,- Global - 010299 01Jan1999.txt,- Global -,010299,01Jan1999,no,NaN,NaN,0.0,NaN,...,True,False,Regional Delegate Programme - appeal 1999 (01....,Annual appeal,- Global - 010299 01Jan1999.pdf,http://www.ifrc.org/docs/appeals/annual99/0102...,OK,none,none,none
4,5,- Global - 010398 01Jan1998.txt,- Global -,010398,01Jan1998,no,NaN,NaN,0.0,NaN,...,True,False,Psychological support programme - appeal 1998 ...,Annual appeal,- Global - 010398 01Jan1998.pdf,http://www.ifrc.org/docs/appeals/annual98/0103...,OK,none,none,none


In [90]:
data.shape

(3997, 25)

In [83]:
data.columns

Index(['Unnamed: 0', 'id.of.documents', 'country', 'appeal.code',
       'date.of.document', 'CTP', 'Number.of.beneficiaries', 'CTP.budget',
       'Number.of.CTP.beneficiaries', 'CTP.budget.source',
       'Budget.relief.items..construction.and.supplies',
       'Budget.relief.fields.names', 'CTP.extract.1', 'CTP.extract.2',
       'CTP.extract.3', 'Appeal', 'DREF', 'name', 'document.name', 'file.name',
       'url', 'url.status', 'glide.number', 'type.of.event', 'date.of.event'],
      dtype='object')

In [84]:
data["file.name"].head(8)

0     - Global - 010198 01Jan1998.pdf
1     - Global - 010199 01Jan1999.pdf
2     - Global - 010298 01Jan1998.pdf
3     - Global - 010299 01Jan1999.pdf
4     - Global - 010398 01Jan1998.pdf
5     - Global - 010498 01Jan1998.pdf
6    - Global - 0110003 01Sep2003.pdf
7    - Global - 0110004 01Jan2004.pdf
Name: file.name, dtype: object

In [91]:
data_CTP_yes = data.loc[(data["CTP"]=="yes")];new_CTP.head(2)

,Unnamed: 0,id.of.documents,country,appeal.code,date.of.document,CTP,Number.of.beneficiaries,CTP.budget,Number.of.CTP.beneficiaries,CTP.budget.source,...,Appeal,DREF,name,document.name,file.name,url,url.status,glide.number,type.of.event,date.of.event
97,98,- Global - MAA00010 02Jul2013.txt,- Global -,MAA00010,02Jul2013,yes,NaN,NaN,0.0,NaN,...,False,True,Disaster Relief Emergency Fund (MAA00010),DREF 2012 Annual Report,- Global - MAA00010 02Jul2013.pdf,http://adore.ifrc.org/Download.aspx?FileId=44833,OK,none,none,none
98,99,- Global - MAA00010 02May2014.txt,- Global -,MAA00010,02May2014,yes,1.0,NaN,0.0,NaN,...,False,True,Disaster Relief Emergency Fund (MAA00010),DREF 2013 Annual Report,- Global - MAA00010 02May2014.pdf,http://adore.ifrc.org/Download.aspx?FileId=58559,OK,none,none,none


In [92]:
data_CTP_yes.shape

(404, 25)

In [94]:
sum(data_CTP_yes.CTP.isnull())

0

In [100]:
real_data = os.chdir("/home/alice/Documents/Project/info_extraction/scraped_data/files_in_txt/")

In [106]:
real_data =  pd.read_csv('/home/alice/Documents/Project/info_extraction/listoftxt_files.csv')

In [110]:
real_data.head()

,SerialNumber,File_names
0,0,"'Tanzania MDRTZ016 10Mar2015.txt',"
1,1,"'Somalia MDRSO003 24Apr2015.txt',"
2,2,"'Afghanistan MDRAF003 05Nov2015.txt',"
3,3,"'Philippines 0 18Dec2015.txt',"
4,4,"'Peru MAAPE002 01May2015.txt',"


In [112]:
len(real_data['File_names'])

1261

In [120]:
cnt = 0
k =[]
for i, f in enumerate(data["file.name"]):
    #print(i, f)
    if f in real_data.File_names.values:
        cnt += 1
    k.append(cnt)     
print(cnt)

393


## Train data set

1.  The csv file  "doc information all_s BIS_r2.csv" has 25 features       for differents   3997  names of files.
2.  Among 3997 reports, 404 are CTP files  and real reports downloaded      by R script are 393

### Due to the short time,  I didnt  extract entities from these files,  probably  It will be my next task after  submission of thesis. 